In [ ]:
import awswrangler as wr
import pandas as pd

AMBIENTE = "qa"
commercial_year = '2021'

In [ ]:
insolvencia_df2 = wr.s3.read_parquet(
        f"s3://cchc-dw-{AMBIENTE}-staging/boletin_concursal/ultima_insolvencia/"
)
sii_df2 = wr.s3.read_parquet(
    f"s3://cchc-dw-{AMBIENTE}-staging/sii/nomina_empresas/commercial_year={commercial_year}/",
    dataset=True,
)
socios_df2 = wr.s3.read_parquet(
    f"s3://cchc-dw-{AMBIENTE}-analytics/dim_socios/", dataset=True , columns=['rut']
)
empresas_df2 = wr.s3.read_parquet(
    f"s3://cchc-dw-{AMBIENTE}-analytics/dim_empresas/", dataset=True
)
input_empresas_manual_df2 = wr.s3.read_parquet(
    f"s3://cchc-dw-{AMBIENTE}-staging/gdrive/sii/empresas_faltantes/existente=False"
)

In [ ]:
insolvencia_df = insolvencia_df2.copy()
sii_df = sii_df2.copy()
socios_df = socios_df2.copy()
empresas_df = empresas_df2.copy()
input_empresas_manual_df = input_empresas_manual_df2.copy()

input_empresas_manual_df["commercial_year"] = commercial_year
sii_df = pd.concat([input_empresas_manual_df, sii_df])

mapping_construccion = {
    "ACTIVIDADES ESPECIALIZADAS DE CONSTRUCCIÓN": [
        "DEMOLICION Y PREPARACION DEL TERRENO",
        "INSTALACIONES ELECTRICAS, DE GASFITERIA Y OTRAS INSTALACIONES PARA OBRAS DE CONSTRUCCION",
        "OTRAS ACTIVIDADES ESPECIALIZADAS DE CONSTRUCCION",
        "TERMINACION Y ACABADO DE EDIFICIOS",
    ],
    "CONSTRUCCIÓN DE EDIFICIOS": ["CONSTRUCCION DE EDIFICIOS"],
    "OBRAS DE INGENIERÍA CIVIL": [
        "CONSTRUCCION DE CARRETERAS Y LINEAS DE FERROCARRIL",
        "CONSTRUCCION DE OTRAS OBRAS DE INGENIERIA CIVIL",
        "CONSTRUCCION DE PROYECTOS DE SERVICIO PUBLICO",
    ],
}

data_mc = []
for k, v in mapping_construccion.items():
    for rubro in v:
        data_mc.append({"grupo_caenes": k, "subrubro_economico": rubro})

mapping_construccion_df = pd.DataFrame(data_mc)

"""
1.- 0< y <= 10UF
2.- 10 < y <= 25UF
3.- 25 < y <= 50UF
4.- 50 < y  <= 100UF
5.- 100 < y <= 250UF
6.- 250 < y <= 500UF
7.- 500 < Y <= 1000UF
8.- 1000 < Y <= 2000UF
9.- 2000 < Y <= 10000UF
10.- 10000UF <
Sin Información
"""

mapping_capital_profio_df = pd.DataFrame(
    [
        {
            "tramo_capital_propio_positivo": 1,
            "tramo_capital_propio_txt": "1. 0< y <= 10UF",
        },
        {
            "tramo_capital_propio_positivo": 2,
            "tramo_capital_propio_txt": "2. 10 < y <= 25UF",
        },
        {
            "tramo_capital_propio_positivo": 3,
            "tramo_capital_propio_txt": "3. 25 < y <= 50UF",
        },
        {
            "tramo_capital_propio_positivo": 4,
            "tramo_capital_propio_txt": "4. 50 < y  <= 100UF",
        },
        {
            "tramo_capital_propio_positivo": 5,
            "tramo_capital_propio_txt": "5. 100 < y <= 250UF",
        },
        {
            "tramo_capital_propio_positivo": 6,
            "tramo_capital_propio_txt": "6. 250 < y <= 500UF",
        },
        {
            "tramo_capital_propio_positivo": 7,
            "tramo_capital_propio_txt": "7. 500 < Y <= 1000UF",
        },
        {
            "tramo_capital_propio_positivo": 8,
            "tramo_capital_propio_txt": "8. 1000 < Y <= 2000UF",
        },
        {
            "tramo_capital_propio_positivo": 9,
            "tramo_capital_propio_txt": "9. 2000 < Y <= 10000UF",
        },
        {
            "tramo_capital_propio_positivo": 10,
            "tramo_capital_propio_txt": "10. 10000UF <",
        },
    ]
)

insolvencia_df["insolvente"] = True
sii_df.rut = sii_df.rut.str.lower()
insolvencia_df.rut = insolvencia_df.rut.str.lower()
socios_df.rut = socios_df.rut.str.lower()
empresas_df.rut = empresas_df.rut.str.lower()
empresas_df["socio_cchc"] = True

df = socios_df.merge(sii_df, how="left", on="rut")

df = df.merge(insolvencia_df[["insolvente", "rut"]], how="left", on="rut")

df = df.merge(
    empresas_df[["rut", "estado", "camara_casa_matriz", "id", "socio_cchc"]],
    "left",
    "rut",
)

df = df[
    [
        df.commercial_year.name,
        df.rut.name,
        df.razon_social.name,
        df.rubro_economico.name,
        df.subrubro_economico.name,
        df.tramo_de_facturacion_txt.name,
        df.numero_de_trabajadores_dependientes_informados.name,
        df.tipo_de_contribuyente.name,
        df.comuna.name,
        df.provincia.name,
        df.region.name,
        df.tamano_de_empresa.name,
        df.insolvente.name,
        df.estado.name,
        df.camara_casa_matriz.name,
        df.socio_cchc.name,
        df.tramo_capital_propio_positivo.name,
        df.tramo_trabajadores.name,
    ]
]

df = df.merge(mapping_construccion_df, "left", "subrubro_economico")

df = df.merge(mapping_capital_profio_df, "left", "tramo_capital_propio_positivo")

df = df.fillna(
    {
        "insolvente": False,
        "socio_cchc": False,
        "tramo_capital_propio_txt": "Sin Información",
    }
)

"""
rut, razon social, grupo caenes, rubro economico, sub rubro economico, tipo contribuyente, tramo facturacion txt, tamaño de empresa (trabajadores), tramo capital propio txt, socio cchc, estado, casa matriz, insolvente
"""
df = df[
    [
        "rut",
        "razon_social",
        "grupo_caenes",
        "rubro_economico",
        "subrubro_economico",
        "tipo_de_contribuyente",
        "tramo_de_facturacion_txt",
        "tamano_de_empresa",
        "tramo_trabajadores",
        "tramo_capital_propio_txt",
        "socio_cchc",
        "estado",
        "camara_casa_matriz",
        "insolvente",
        "commercial_year",
    ]
]

df = df.fillna(
    {
        "tramo_trabajadores": "Sin Información",
        "region": "Sin Información",
        "tramo_de_facturacion_txt": "No Informado",
        "rubro_economico": "Valor por Defecto",
        "subrubro_economico": "Valor por Defecto",
    }
)

df.commercial_year = pd.to_numeric(commercial_year)
df.commercial_year = df.commercial_year.astype("Int32")

len(df)

In [ ]:
prueba = socios_df[socios_df.rut == '7217956-0']
prueba

In [ ]:
wr.s3.to_parquet(
    df=df,
    path=f"s3://cchc-dw-{AMBIENTE}-analytics/sii_socios_insolvencia",
    dataset=True,
    table="sii_socios_insolvencia",
    database="analytics_" + AMBIENTE,
    mode="overwrite",
    partition_cols=["rubro_economico", "commercial_year"],
    compression="gzip",
)